# Cleaning

#### NA's (or NaNs) as value
1. **Alley:** Type of alley access to property
2. **BsmtQual:** Evaluates the height of the basement
3. **BsmtCond:** Evaluates the general condition of the basement
4. **BsmtExposure:** Refers to walkout or garden level walls
5. **BsmtFinType1:** Rating of basement finished area
6. **BsmtFinType2:** Rating of basement finished area (if multiple types)*
7. **FireplaceQu:** Fireplace quality
8. **GarageType:** Garage location
9. **GarageFinish:** Interior finish of the garage
10. **GarageQual:** Garage quality
11. **GarageCond:** Garage condition
12. **PoolQC:** Pool quality (Biggest of the above)
13. **Fence:** Fence quality
14. **MiscFeature:** Miscellaneous feature not covered in other categories

##### The two below are listed as None in description, but NA in dataset
* MasVnrArea: Masonry veneer type
* MasVnrType: Masonry veneer area in square feet

##### Others
* LotFrontage: Maybe set to NA if there is no street connected to property?
* GarageYrBlt: Set to NA if the above Garage attributes are set as NA

* Electrical: Electrical system. There is only one property with a missing data in Electrical. At row 1381


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ds_utils import *
import copy
%matplotlib inline

In [2]:
DFtrain = pd.read_csv('train.csv')
Y = DFtrain['SalePrice']
DFtest = pd.read_csv('test.csv')
DFtrain = DFtrain.drop(['SalePrice'], axis=1)
# DFtrain.shape, DFtest.shape

In [3]:
DFc = pd.concat([DFtrain, DFtest])
del DFc['Id']
del DFc['GarageYrBlt']
del DFc['LotShape']
DFc.shape

(2919, 77)

In [4]:
def summary_missing_data(data, n):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data.head(n)

In [5]:
missing_data = summary_missing_data(DFc,34)
print(missing_data)

              Total   Percent
PoolQC         2909  0.996574
MiscFeature    2814  0.964029
Alley          2721  0.932169
Fence          2348  0.804385
FireplaceQu    1420  0.486468
LotFrontage     486  0.166495
GarageCond      159  0.054471
GarageQual      159  0.054471
GarageFinish    159  0.054471
GarageType      157  0.053786
BsmtCond         82  0.028092
BsmtExposure     82  0.028092
BsmtQual         81  0.027749
BsmtFinType2     80  0.027407
BsmtFinType1     79  0.027064
MasVnrType       24  0.008222
MasVnrArea       23  0.007879
MSZoning          4  0.001370
BsmtHalfBath      2  0.000685
Utilities         2  0.000685
Functional        2  0.000685
BsmtFullBath      2  0.000685
BsmtFinSF2        1  0.000343
BsmtFinSF1        1  0.000343
BsmtUnfSF         1  0.000343
TotalBsmtSF       1  0.000343
Exterior2nd       1  0.000343
SaleType          1  0.000343
Exterior1st       1  0.000343
Electrical        1  0.000343
KitchenQual       1  0.000343
GarageArea        1  0.000343
GarageCars

#### Getting rid of rows with few missinng data

In [6]:
for value in ((missing_data[missing_data['Total'] < 50]).index):
    DFc = DFc.drop(DFc.loc[DFc[value].isnull()].index)
summary_missing_data(DFc,17)

,Total,Percent
PoolQC,2835,0.996485
MiscFeature,2742,0.963796
Alley,2651,0.931810
Fence,2280,0.801406
FireplaceQu,1387,0.487522
LotFrontage,466,0.163796
GarageFinish,156,0.054833
GarageCond,156,0.054833
GarageQual,156,0.054833
GarageType,155,0.054482


In [7]:
DFc.shape

(2845, 77)

#### Filled missing values in LotFrontage with the mode

In [8]:
LotFrontage = DFc['LotFrontage']
mode = DFc['LotFrontage'].mode() # Mode is 60
mode = float(mode)
DFc['LotFrontage'].fillna(mode, inplace=True)

#### Filled the remaining missing values with "None"
The remaining ones just have None listed as NA in the data description

In [9]:
DFc.fillna("None",inplace=True)
summary_missing_data(DFc,5)

,Total,Percent
SaleCondition,0,0.0
Foundation,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0


In [10]:
atts = pd.read_csv('attributes.csv')

In [11]:
DFdum = DFc.copy()

### Trying to get dummies only for Categorical atts

In [12]:
for i in range(0, len(atts)):
    if atts.iloc[i]['Type']=='Categorical':
        col_name=atts.iloc[i]['Attribute']
        DFc=pd.get_dummies(DFc, columns=[col_name])

In [13]:
DFc

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,MiscVal_17000,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,65.0,8450,2003,2003,196.0,706.0,0.0,150.0,856.0,856,...,0,0,0,0,0,0,0,0,0,1
1,80.0,9600,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,...,0,0,0,0,0,0,0,0,0,1
2,68.0,11250,2001,2002,162.0,486.0,0.0,434.0,920.0,920,...,0,0,0,0,0,0,0,0,0,1
3,60.0,9550,1915,1970,0.0,216.0,0.0,540.0,756.0,961,...,0,0,0,0,0,0,0,0,0,1
4,84.0,14260,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,...,0,0,0,0,0,0,0,0,0,1
5,85.0,14115,1993,1995,0.0,732.0,0.0,64.0,796.0,796,...,0,0,0,0,0,0,0,0,0,1
6,75.0,10084,2004,2005,186.0,1369.0,0.0,317.0,1686.0,1694,...,0,0,0,0,0,0,0,0,0,1
7,60.0,10382,1973,1973,240.0,859.0,32.0,216.0,1107.0,1107,...,0,0,0,0,0,0,0,0,0,1
8,51.0,6120,1931,1950,0.0,0.0,0.0,952.0,952.0,1022,...,0,0,0,0,0,0,0,0,0,1
9,50.0,7420,1939,1950,0.0,851.0,0.0,140.0,991.0,1077,...,0,0,0,0,0,0,0,0,0,1


# Modeling

Was going to get ready for PCA stuff below

In [14]:
from sklearn.preprocessing import StandardScaler

y = DF.loc[:,['SalePrice']].values
DF = DF.drop(['SalePrice'], axis=1)
y

X = datatrain.drop(columns=['SalePrice', 'Id'])
Y = datatrain['SalePrice']
Xt = datatest.drop(columns=['Id'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Xt_scaled = scaler.fit_transform(Xt)

